In [2]:
!pip install bm3d



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.0/862.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 23.1 MB/s eta 0:00:00


In [6]:
# Import required libraries
import os
import csv
import bm3d
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from google.colab import drive

# Mount Google Drive to access images
drive.mount('/content/drive')

# Set input folder
input_folder = "/content/drive/MyDrive/cbsd68/original_png"  # Folder with input images
base_output_folder = "/content/drive/MyDrive/cbsd68/denoised_images"  # Base folder for denoised images
os.makedirs(base_output_folder, exist_ok=True)

# List of sigma values
sigma_values = [25, 50, 75]
batch_size = 8  # Number of images to process in each batch

def compute_metrics(clean_img, denoised_img):
    """Compute PSNR and SSIM metrics."""
    psnr_value = psnr(clean_img, denoised_img, data_range=1.0)

    min_size = min(clean_img.shape[:2])  # Adjust win_size dynamically
    win_size = min(7, min_size)
    if win_size % 2 == 0:
        win_size -= 1  # Ensure win_size is odd
    ssim_value = ssim(clean_img, denoised_img, channel_axis=2, win_size=win_size, data_range=1.0)

    return psnr_value, ssim_value

# Load all image paths from the input folder
image_paths = [os.path.join(input_folder, fname) for fname in os.listdir(input_folder) if fname.endswith(('.png', '.jpg', '.jpeg'))]

# Process images for each sigma value
for sigma in sigma_values:
    # Create a folder for the current sigma value
    output_folder = os.path.join(base_output_folder, f"sigma_{sigma}")
    os.makedirs(output_folder, exist_ok=True)

    # CSV file to store PSNR and SSIM for the current sigma
    csv_path = os.path.join(output_folder, "image_metrics.csv")
    csv_file = open(csv_path, mode='w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Image Name", "PSNR (dB)", "SSIM"])

    sigma_psd = sigma / 255.0  # Normalize sigma

    # Batch processing
    for batch_start in range(0, len(image_paths), batch_size):
        batch_end = min(batch_start + batch_size, len(image_paths))
        batch = image_paths[batch_start:batch_end]

        try:
            clean_imgs = []
            noisy_imgs = []

            # Prepare images for the batch
            for image_path in batch:
                img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Read as BGR
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0  # Convert to RGB and normalize
                noisy_img = img_rgb + np.random.normal(0, sigma_psd, img_rgb.shape)
                noisy_img = np.clip(noisy_img, 0, 1)

                clean_imgs.append(img_rgb)
                noisy_imgs.append(noisy_img)

            # Apply BM3D to each image in the batch
            denoised_imgs = []
            for noisy_img in noisy_imgs:
                denoised_imgs.append(bm3d.bm3d(noisy_img, sigma_psd))

            # Compute PSNR and SSIM for the batch
            for i, (clean_img, denoised_img) in enumerate(zip(clean_imgs, denoised_imgs)):
                psnr_value, ssim_value = compute_metrics(clean_img, denoised_img)

                # Save denoised image
                denoised_img_uint8 = (denoised_img * 255).astype(np.uint8)
                image_name = f"denoised_{batch_start + i:05d}.png"
                save_path = os.path.join(output_folder, image_name)
                cv2.imwrite(save_path, cv2.cvtColor(denoised_img_uint8, cv2.COLOR_RGB2BGR))

                # Write PSNR and SSIM to CSV
                csv_writer.writerow([image_name, psnr_value, ssim_value])

                print(f"Processed image {batch_start + i + 1}/{len(image_paths)} for sigma={sigma}: PSNR={psnr_value:.2f} dB, SSIM={ssim_value:.4f}")

        except Exception as e:
            print(f"Error processing batch {batch_start} to {batch_end} for sigma={sigma}: {e}")
            continue

    # Close the CSV file for the current sigma
    csv_file.close()
    print(f"Results for sigma={sigma} saved to: {output_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed image 1/69 for sigma=25: PSNR=32.66 dB, SSIM=0.8566
Processed image 2/69 for sigma=25: PSNR=28.55 dB, SSIM=0.8111
Processed image 3/69 for sigma=25: PSNR=27.47 dB, SSIM=0.8393
Processed image 4/69 for sigma=25: PSNR=29.03 dB, SSIM=0.7842
Processed image 5/69 for sigma=25: PSNR=28.61 dB, SSIM=0.8072
Processed image 6/69 for sigma=25: PSNR=27.83 dB, SSIM=0.8219
Processed image 7/69 for sigma=25: PSNR=28.95 dB, SSIM=0.8928
Processed image 8/69 for sigma=25: PSNR=28.18 dB, SSIM=0.8507
Processed image 9/69 for sigma=25: PSNR=24.80 dB, SSIM=0.8172
Processed image 10/69 for sigma=25: PSNR=29.15 dB, SSIM=0.7558
Processed image 11/69 for sigma=25: PSNR=32.12 dB, SSIM=0.8893
Processed image 12/69 for sigma=25: PSNR=23.85 dB, SSIM=0.7635
Processed image 13/69 for sigma=25: PSNR=29.52 dB, SSIM=0.7840
Processed image 14/69 for sigma=25: PSNR=27.76 dB, SSIM=0.787